In [2]:
import sys
sys.path.append("../MMPF")
sys.path.append("../scripts")

import data
import utils
import experiments_new
import models


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
dataset = "acsincome"
X_train, Y_train, X_val, Y_val, X_test, Y_test =  data.get_fold(dataset, 5, 10, 0)
A_train, A_val, A_test = experiments_new.get_subgroup_feature(dataset, X_train, X_val, X_test, 4)
col_trans = ColumnTransformer(
    [
        ("numeric", StandardScaler(), data.NUM_FEATURES[dataset]),
        (
            "categorical",
            OneHotEncoder(
                drop="if_binary", sparse_output=False, handle_unknown="ignore"
            ),
            data.CAT_FEATURES[dataset],
        ),
    ],
    verbose_feature_names_out=False,
)
col_trans.set_output(transform="pandas")

preprocess = Pipeline([("preprocess", col_trans)])
preprocess.fit(X_train)
X_train = preprocess.transform(X_train)
X_val = preprocess.transform(X_val)
X_test = preprocess.transform(X_test)

{'1': 0, '2': 1, '4': 2, '3': 3}


In [ ]:
m = models.MinimaxPareto(n_iterations = 10, max_iter = 10)
m.fit(X_train, Y_train, A_train)